# First Exploration of Synthesized beams
### JCH - March 2025

The idea is to explore the various synthesized beam modelings that are available in the Qubic code. We will investigate the implementation of the following models:
- Theoretical synthesized beam models (e.g. Gaussian approximations, Physical-Optics models, etc.)
- Implementation of measured synthesized beams models in the code (as done by Michael Wright)

We will also overview the various `CalFiles` associated with these synthesized beams models and update them to the most recent knowledge.

Here's the task that was decided and assigned to JCH during the APC D/A meeting on March 11th 2025:
```
code from Michael Wright about the use of calibration files. This code is mainly the following files : Qinstrument.py, Qbeams.py, Qcalibration.py, maybe others. There is different things to do, I will try to list what I found so far:
- Verify the code
- Avoid running the calibration file code when you haven't select the parameter to use them
- Long term task: test the calibration files that are currently in the qubicsoft, and start to discuss about making additional ones
```

In [ ]:
# Mandatory imports
%config InlineBackend.figure_format='retina'
from IPython.display import display, HTML

%matplotlib widget
%load_ext autoreload
%autoreload 2

import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from astropy.io import fits


from qubic.lib.Qdictionary import qubicDict
from qubic.lib.Instrument.Qinstrument import QubicMultibandInstrument
from qubic.lib.Qscene import QubicScene

plt.rcParams['figure.figsize'] = (8,4)

def print_keys(d, keys):
    for key in keys:
        print(' - {:25}: {}'.format(key, d[key]))
    print('\n')

# Analytical beam positions
We first use the `_peak_angles()` function from `QubicMultibandInstrument` to get the analytical positions and apmlitudes of the peaks in the synthesized beam and we compare them to the actual synthesized beam image obtained also analyticall from `get_synthbeam()` function (also from `QubicMultibandInstrument`). We should get exact results (or almost, the image is pixellised so some small effect are to be expected, but should reduce with higher `nside`).

In order to do so, we set:
```
d['synthbeam'] = None         # we put nothing
d['use_synthbeam_fits_file'] = False
```
to tell the code not to attempt to read a CalFile for the synthesized beam and make sure it uses the theoretical formulas.

It is important to check for the various cases here:
- TD and FI
- 150 and 220 GHz bands


### In order to make this work:
JCH has modified `__init__()` in `QubicMultibandInstrument` moving the `self.d=d1` to the inside of the loop over sub_instruments so that they each have the right frequency in `q_instrument.subinstruments[i].d['filter_nu']`.

In [ ]:
dictfilename = 'qubic/qubic/dicts/pipeline_demo.dict'

d = qubicDict()
d.read_from_file(dictfilename)

print('After reading dict: d[filter_nu] = ', d['filter_nu'])
print()


d['debug'] = True
d['config'] = 'TD'
d['instrument_type'] = 'MB'
d['MultiBand'] = True
d['nf_sub'] = 3
d['beam_shape'] = 'gaussian'  # can be 'gaussian', 'fitted_beam' or 'multi_freq'  
d['nside'] = 512*4              # To have nice SB maps
d['synthbeam_fraction'] = 1
d['synthbeam_kmax'] = 2

d['synthbeam'] = None         # we put nothing
d['use_synthbeam_fits_file'] = False

### Creating QubicMultibandInstrument
q_instrument = QubicMultibandInstrument(d)
print('###################################')
print('Checking sub-nsitrments frequencies')
print('###################################')
for i in range(d['nf_sub']):
    print('SubInst {} d[filter_nu] = {:6.2f} GHz'.format(i, q_instrument.subinstruments[i].d['filter_nu']/1e9))
print('###################################')
print('\n\n\n')


### Now setting the cene
q_scene = QubicScene(d)



## middle frequency SB peak positions
imed = d['nf_sub']//2
medfreq = q_instrument.subinstruments[imed].d['filter_nu']
print('Middle frequency bin ({:6.2f} GHz) SB peak positions'.format(medfreq/1e9))
print('i:       Theta   Phi     Amp')
thetas, phis, vals = q_instrument[imed]._peak_angles(q_scene, 
                                               medfreq, 
                                               q_instrument[imed].detector.center, 
                                               q_instrument[imed].synthbeam, 
                                               q_instrument[imed].horn, 
                                               q_instrument[imed].primary_beam)

idet = np.random.randint(248)
for i in range(np.shape(thetas)[1]):
    print('{0:5.0f}: {1:7.3g} {2:7.3g} {3:7.3g}'.format(i, thetas[idet,i], phis[idet, i], vals[idet, i]/np.max(vals[idet,:])))

# ######### Calculate full synthesized beam map
sb = q_instrument[imed].get_synthbeam(q_scene, idet)
plt.figure()
plt.subplot(1,2,1)
pixnum = hp.ang2pix(d['nside'], thetas[idet,:], phis[idet,:])
newvals = (sb/np.max(sb))[pixnum]
plt.plot(vals[idet]/np.max(vals[idet,:]), 100*(vals[idet,:]/np.max(vals[idet,:])/newvals-1), 'ro')
ss = np.nanstd(vals[idet,:]/np.max(vals[idet,:])/newvals)
plt.ylim(-3*ss*100, 3*ss*100)
plt.axhline(y=0, ls=':', color='k')
plt.xscale('log')
plt.xlabel('Peak_angle amplitudes')
plt.ylabel('Relative difference w.r.t. SB ma (%)')
plt.title('Theory {}\n Mid-Freq Bin {:6.2f} GHz: TES #{}'.format(d['config'], medfreq/1e9,idet))


hp.gnomview(np.log10(sb/np.max(sb)), rot=[0,90], reso=15, min=-5, max=0, sub=(1,2,2),
            title='Theory {0:} {1:7.2f} GHz: TES #{2:}'.format(d['config'], d['filter_nu']/1e9,idet))
hp.projscatter(thetas[idet,:], phis[idet,:], c=vals[idet,:]/np.max(vals[idet,:]), marker='x', cmap='Reds', label='From _peak_angles()')
for i in range(np.shape(thetas)[1]):
    hp.projtext(thetas[idet,i], phis[idet,i], 
                '{0:5.0f}: {1:4.2f}'.format(i, vals[idet,i]/np.max(vals[idet,:])), c='w', fontsize=8)
plt.legend(loc='upper right')
plt.tight_layout()

# Now evolution with frequency

In [ ]:
################## Setting dictionnary #######################
dictfilename = 'qubic/qubic/dicts/pipeline_demo.dict'
d = qubicDict()
d.read_from_file(dictfilename)

### Instrument type
d['config'] = 'TD'             # can be 'TD' or 'FI'
d['instrument_type'] = 'MB'    # can be 'DB', 'UWB' or 'MB'

### SB related stuff
#mykeys = ['beam_shape', 'synthbeam', 'use_synthbeam_fits_file']
d['beam_shape'] = 'gaussian'  # can be 'gaussian', 'fitted_beam' or 'multi_freq'  
d['synthbeam'] = None         # we put nothing
d['use_synthbeam_fits_file'] = False
d['synthbeam_kmax'] = 2
d['use_synthbeam_fits_file'] = False

### Frequency description
d['nf_sub'] = 11

### Other paramters
d['nside'] = 512              # To have nice SB maps

if d['config'] == 'TD':
    Ndet = 248
elif d['config'] == 'FI':
    Ndet = 992
else:
    print('Wrong config in dict')
    stop

################# Instanciating instrument ###################
d['debug'] = False
q_instrument = QubicMultibandInstrument(d)
q_scene = QubicScene(d)

print()
for i in range(d['nf_sub']):
    print('Sub instrument {} nu={:6.2f} GHz dnu_nu/nu = {} dnu = {}'.format(i, 
                                                                   q_instrument.subinstruments[i].d['filter_nu']/1e9, 
                                                                   q_instrument.subinstruments[i].FRBW, 
                                                                   q_instrument.subinstruments[i].d["filter_relative_bandwidth"]))
print()


idet = np.random.randint(Ndet)
thetas = np.zeros((d['nf_sub'], Ndet, (2*d['synthbeam_kmax']+1)**2))
phis = np.zeros((d['nf_sub'], Ndet, (2*d['synthbeam_kmax']+1)**2))
vals = np.zeros((d['nf_sub'], Ndet, (2*d['synthbeam_kmax']+1)**2))
my_sbs = np.zeros((d['nf_sub'], 12*d['nside']**2))
average_sb = np.zeros(12*d['nside']**2)
for i in range(d['nf_sub']):
    thetas[i,:,:], phis[i,:,:], vals[i,:,:]  = q_instrument.subinstruments[i]._peak_angles(q_scene, 
                                               q_instrument.subinstruments[i].d['filter_nu'], 
                                               q_instrument.subinstruments[i].detector.center, 
                                               q_instrument.subinstruments[i].synthbeam, 
                                               q_instrument.subinstruments[i].horn, 
                                               q_instrument.subinstruments[i].primary_beam)

    print('subinstrument',i)
    print(q_instrument[i].d['filter_nu'])
    print(thetas[i,idet,:])
    ######### Calculate full synthesized beam map
    my_sbs[i,:] = q_instrument[i].get_synthbeam(q_scene, idet)
    plt.figure()
    plt.subplot(1,2,1)
    pixnum = hp.ang2pix(d['nside'], thetas[i, idet,:], phis[i, idet,:])
    newvals = (my_sbs[i,:]/np.max(my_sbs[i,:]))[pixnum]
    plt.plot(vals[i, idet]/np.max(vals[i, idet,:]), 100*(vals[i, idet,:]/np.max(vals[i, idet,:])/newvals-1), 'ro')
    ss = np.nanstd(vals[i, idet,:]/np.max(vals[i, idet,:])/newvals)
    plt.ylim(-3*ss*100, 3*ss*100)
    plt.axhline(y=0, ls=':', color='k')
    plt.xscale('log')
    plt.xlabel('Peak_angle amplitudes')
    plt.ylabel('Relative difference w.r.t. SB ma (%)')
    plt.title('Theory {0:}\n Sub {1:}/{2:}: {3:7.2f} GHz: TES #{4:}'.format(d['config'], i, d['nf_sub'], q_instrument.subinstruments[i].d['filter_nu']/1e9,idet))


    hp.gnomview(np.log10(my_sbs[i,:]/np.max(my_sbs[i,:])), rot=[0,90], reso=15, min=-5, max=0, sub=(1,2,2),
              title='Theory {0:} {1:7.2f} GHz: TES #{2:}'.format(d['config'], q_instrument.subinstruments[i].d['filter_nu']/1e9,idet))
    hp.projscatter(thetas[i, idet,:], phis[i, idet,:], c=vals[i, idet,:]/np.max(vals[i, idet,:]), 
                   marker='x', cmap='Reds', label='From _peak_angles()')
    plt.legend(loc='upper right')
    plt.tight_layout()
    average_sb += my_sbs[i,:] / d['nf_sub']


imed = d['nf_sub']//2
hp.gnomview(np.log10(my_sbs[imed,:]/np.max(my_sbs[imed,:])), rot=[0,90], reso=15, min=-5, max=0,
            title='Theory {0:} {1:7.2f} GHz: TES #{2:}'.format(d['config'], q_instrument.subinstruments[imed].d['filter_nu']/1e9,idet))
hp.gnomview(np.log10(my_sbs[imed,:]/np.max(my_sbs[imed,:])), rot=[0,90], reso=15, min=-5, max=0,
            title='Theory {0:} {1:7.2f} GHz: TES #{2:}'.format(d['config'], q_instrument.subinstruments[imed].d['filter_nu']/1e9,idet))
for i in range(d['nf_sub']):
    hp.projscatter(thetas[i, idet,:], phis[i, idet,:], c=vals[i, idet,:]/np.max(vals[i, idet,:]), 
                    marker='x', cmap='Reds', label='From _peak_angles()')
plt.tight_layout()


hp.gnomview(np.log10(average_sb/np.max(average_sb)), rot=[0,90], reso=15, min=-5, max=0,
            title='Average SB [Log]')
hp.gnomview(average_sb/np.max(average_sb), rot=[0,90], reso=15, min=0, max=1,
            title='Average SB')



# Calibration Files

In [ ]:
### Reading the files directly
def read_myfile(filename):
    hdu = fits.open(filename)
    header = hdu[0].header
    theta = hdu[0].data
    phi = hdu[1].data
    val = hdu[2].data
    freqs = hdu[3].data
    return theta, phi, val, freqs, header


cal_directory = os.path.dirname(q_instrument[0].calibration.detarray)
allfiles = glob.glob(cal_directory+'/*Synthbeam*.fits')
for f in allfiles:
    print()
    print('#######################################################')
    print(os.path.basename(f))
    print('-------------------------------------------------------')
    theta, phi, val, freqs, header = read_myfile(f)
    print('---- HEADER -------------------------------------------')
    print(header)
    print('---- NFREQ --------------------------------------------')
    print('    Nfreq = {}'.format(len(freqs)))
    print('   ', freqs)
    print('---- THETAS -------------------------------------------')
    print('   Thetas: ',np.shape(theta))
    print('---- PHIS ---------------------------------------------')
    print('     Phis:',np.shape(phi))
    print('---- VALS ---------------------------------------------')
    print('     Vals:',np.shape(val))
    print('#######################################################')


OK, so we see that nost files are absurd. They need to be redone. The last one (CalQubic_Synthbeam_Analytical_Multifreq_MJW_FI.fits) seems to have the right structure although the frequencies are absurd (220 popping in for no reason !):
```
CalQubic_Synthbeam_Analytical_Multifreq_MJW_FI.fits
    Nfreq = 11
    [133 136 137 141 146 148 151 156 161 166 220]
   Thetas:  (11, 992, 9)
     Phis: (11, 992, 9)
     Vals: (11, 992, 9)
```
So we will take this as an example format and fill it in another notebook.